In [22]:
import os
import pandas as pd
import numpy as np
#from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array

# Load data

In [27]:
image_folder = "/Users/liliane.bader/Documents/AU/DLfVR/project/data/archive/HAM10000_images_part_1/"
metadata_path = "/Users/liliane.bader/Documents/AU/DLfVR/project/data/archive/HAM10000_metadata.csv"

metadata = pd.read_csv(metadata_path)
metadata['image_path'] = metadata['image_id'].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))
metadata_part1 = metadata[(metadata['image_id'] > 'ISIC_0024305') & (metadata['image_id'] < 'ISIC_0029306')]

# Load images and preprocess them
image_size = (64, 64)  

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img = img_to_array(img) / 255.0  # Normalize to [0,1]
    return img

# Load all images into a list and convert to a numpy array
images_part1 = np.array([load_and_preprocess_image(img_path) for img_path in metadata_part1['image_path']])
print(images_part1.shape) 

(5000, 224, 224, 3)


In [28]:
labels_part1 = metadata_part1['dx']
label_mapping = {label: idx for idx, label in enumerate(labels_part1.unique())}
metadata_part1.loc[:, 'label'] = metadata_part1['dx'].map(label_mapping)
labels_encoded = to_categorical(metadata_part1['label'])
print(metadata_part1['label'].head(10))
print(metadata_part1['dx'].unique())
print(metadata_part1['label'].value_counts())

0     0
1     0
2     0
3     0
5     0
6     0
7     0
8     0
9     0
10    0
Name: label, dtype: int64
['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
label
1    3431
0     564
3     435
5     266
6     183
4      65
2      56
Name: count, dtype: int64


/var/folders/zt/fk8vkvl57m1ghjg3p98gjq5r0000gp/T/ipykernel_86412/3816237097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_part1.loc[:, 'label'] = metadata_part1['dx'].map(label_mapping)


In [18]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images_part1, labels_encoded, test_size=0.2, random_state=42)


# Define a simple baseline model (CNN)

In [23]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_mapping), activation='softmax')  # Output layer for classification
])

/Users/liliane.bader/Documents/AU/DLfVR/project/deeplproject/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [25]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.6688 - loss: 1.1748 - val_accuracy: 0.6650 - val_loss: 0.9739
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7091 - loss: 0.8875 - val_accuracy: 0.6730 - val_loss: 0.8799
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.7192 - loss: 0.8374 - val_accuracy: 0.6690 - val_loss: 0.9108
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.7046 - loss: 0.8056 - val_accuracy: 0.6880 - val_loss: 0.8548
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7125 - loss: 0.7919 - val_accuracy: 0.6890 - val_loss: 0.8411
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.7341 - loss: 0.7426 - val_accuracy: 0.6910 - val_loss: 0.8218
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.7326 - loss: 0.7338 - val_accuracy: 0.7010 - val_loss: 0.8079
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.7468 - loss: 0.6980 - val_accu

In [26]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7163 - loss: 0.7635
Test Accuracy: 0.7200


# Simple Model using ResNet 

In [29]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [30]:
# Load the metadata
metadata_path = "/Users/liliane.bader/Documents/AU/DLfVR/project/data/archive/HAM10000_metadata.csv"
metadata = pd.read_csv(metadata_path)

# Create a label mapping from 'dx' column
label_mapping = {label: idx for idx, label in enumerate(metadata['dx'].unique())}
metadata['label'] = metadata['dx'].map(label_mapping)

# Check the label distribution
print(metadata['label'].value_counts())

label
1    6705
3    1113
0    1099
5     514
6     327
4     142
2     115
Name: count, dtype: int64


In [36]:
# Image directories
image_folder_1 = "/Users/liliane.bader/Documents/AU/DLfVR/project/data/archive/HAM10000_images_part_1/"
image_folder_2 = "/Users/liliane.bader/Documents/AU/DLfVR/project/data/archive/HAM10000_images_part_2/"

# Combine paths of images from both parts
metadata['image_path'] = metadata['image_id'].apply(lambda x: 
    os.path.join(image_folder_1, x + ".jpg") 
    if os.path.exists(os.path.join(image_folder_1, x + ".jpg"))
    else os.path.join(image_folder_2, x + ".jpg")
)

# Split into train and test sets
train_df, test_df = train_test_split(metadata, stratify=metadata['label'], test_size=0.2)

# Image size for ResNet50
image_size = (224, 224)

# Create ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Convert integer labels to strings
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)


# Flow images from dataframe
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    x_col='image_path', 
    y_col='label', 
    target_size=image_size,
    class_mode= 'categorical',
    batch_size=32
)

test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    x_col='image_path', 
    y_col='label', 
    target_size=image_size, 
    class_mode= 'categorical',
    batch_size=32
)


Found 8012 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


In [37]:
# Load the pre-trained ResNet50 model, excluding the top classification layer
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the ResNet50 base layers so they won't be trained
for layer in resnet_model.layers:
    layer.trainable = False

# Create a new model on top of ResNet50
model = Sequential([
    resnet_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(len(label_mapping), activation='softmax')  # Adjust the number of output classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Check model summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,055 (91.99 MB)

 Trainable params: 526,343 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [38]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,  # Adjust based on your needs
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

Epoch 1/10


/Users/liliane.bader/Documents/AU/DLfVR/project/deeplproject/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.6352 - loss: 1.1538 - val_accuracy: 0.7479 - val_loss: 0.7052
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


2024-10-08 11:35:06.142765: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/liliane.bader/.pyenv/versions/3.9.10/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-10-08 11:35:06.167515: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


251/251 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.7707 - loss: 0.6337 - val_accuracy: 0.7678 - val_loss: 0.6451
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 125us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10


2024-10-08 11:40:14.117268: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


251/251 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.7942 - loss: 0.5548 - val_accuracy: 0.7763 - val_loss: 0.6057
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 90us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.8175 - loss: 0.4918 - val_accuracy: 0.7813 - val_loss: 0.5988
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 99us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10


2024-10-08 11:51:22.414804: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


238/251 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8392 - loss: 0.4455

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


In [ ]:
keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    name="resnet50",
)